In [ ]:
!pip install coral_pytorch transformers datasets torch scikit-learn accelerate

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, mean_absolute_error, mean_squared_error
from transformers import (
    AutoTokenizer,
    AutoModel,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset

from coral_pytorch.losses import CornLoss
from coral_pytorch.dataset import corn_label_from_logits


SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


Libraries imported and seed set.


In [ ]:
MODEL_NAME = "xlm-roberta-base"
N_FOLDS = 5
MAX_LEN = 512
BATCH_SIZE = 8
EPOCHS = 6          
NUM_CLASSES = 8

In [ ]:
class XLMRCornOrdinal(nn.Module):
    def __init__(self, model_checkpoint, num_classes=8):
        super(XLMRCornOrdinal, self).__init__()
        self.backbone = AutoModel.from_pretrained(model_checkpoint)
        hidden_size = self.backbone.config.hidden_size

        # CORN Architecture: (K-1) binary classifiers
        self.corn_layer = nn.Linear(hidden_size, num_classes - 1)

    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        # 1. Get Transformer Embeddings
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]

        # 2. Pass through CORN Layer
        logits = self.corn_layer(cls_embedding)

        loss = None
        if labels is not None:
            loss_fct = CornLoss(num_classes=8)
            loss = loss_fct(logits, labels)

        return (loss, logits) if loss is not None else logits

In [ ]:
df = pd.read_csv('training_data.csv')

In [ ]:
if df['labels'].min() == 1:
    print("Shifting labels from 1-8 to 0-7...")
    df['labels'] = df['labels'] - 1

Shifting labels from 1-8 to 0-7...


In [ ]:
df['labels'] = df['labels'].astype(int)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

In [ ]:
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

best_score = -float('inf')
best_model_dir = "./best_corn_model"

fold_metrics = {'accuracy': [], 'qwk': [], 'mae': [], 'rmse': []}

print(f"Starting {N_FOLDS}-Fold CV (CORN Implementation)...")

Starting 5-Fold CV (CORN Implementation)...


In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['labels'])):
    print(f"\n{'='*20} FOLD {fold+1}/{N_FOLDS} {'='*20}")

    # A. Split
    train_dataset = Dataset.from_pandas(df.iloc[train_idx])
    val_dataset = Dataset.from_pandas(df.iloc[val_idx])

    # B. Tokenize
    train_tokenized = train_dataset.map(tokenize_function, batched=True)
    val_tokenized = val_dataset.map(tokenize_function, batched=True)

    # C. Initializing Model (Fresh for each fold)
    model = XLMRCornOrdinal(MODEL_NAME, NUM_CLASSES)

    # D. Metrics (Special for CORN)
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        # logits shape: [Batch, 7] (for 8 classes)

        # CORN Inference: Convert binary logits to integer rank
        # corn_label_from_logits applies the chain rule logic
        predictions = corn_label_from_logits(torch.tensor(logits)).numpy()

        # Calculate Metrics
        acc = accuracy_score(labels, predictions)
        qwk = cohen_kappa_score(labels, predictions, weights='quadratic')
        mae = mean_absolute_error(labels, predictions)
        rmse = np.sqrt(mean_squared_error(labels, predictions))

        return {
            'accuracy': acc,
            'qwk': qwk,
            'mae': mae,
            'rmse': rmse
        }

    # E. Training Arguments
    training_args = TrainingArguments(
        output_dir=f'./results_corn_fold_{fold+1}',
        num_train_epochs=EPOCHS,        
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=1e-5,               
        weight_decay=0.01,

        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="qwk",
        greater_is_better=True,
        save_total_limit=1,

        logging_steps=50,
        fp16=torch.cuda.is_available(),

        remove_unused_columns=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # F. Train
    trainer.train()
    metrics = trainer.evaluate()

    print(f"Fold {fold+1} Result: QWK={metrics['eval_qwk']:.4f} | MAE={metrics['eval_mae']:.4f}")

    fold_metrics['accuracy'].append(metrics['eval_accuracy'])
    fold_metrics['qwk'].append(metrics['eval_qwk'])
    fold_metrics['mae'].append(metrics['eval_mae'])
    fold_metrics['rmse'].append(metrics['eval_rmse'])

    if metrics['eval_qwk'] > best_score:
        print(f"New Best Score (QWK {metrics['eval_qwk']:.4f})! Saving...")
        best_score = metrics['eval_qwk']
        trainer.save_model(best_model_dir)
        tokenizer.save_pretrained(best_model_dir)

    del model, trainer
    torch.cuda.empty_cache()




print("\n" + "="*40)
print(f"FINAL CORN RESULTS ({N_FOLDS}-Fold CV)")
print("="*40)
for k, v in fold_metrics.items():
    print(f"{k.upper()}: {np.mean(v):.4f} ± {np.std(v):.4f}")


==================== FOLD 1/5 ====================


Map:   0%|          | 0/910 [00:00<?, ? examples/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: xlm-roberta-base
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.weight      | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.dense.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


...

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: xlm-roberta-base
Key                       | Status     |  | 
--------------------------+------------+--+-
lm_head.dense.weight      | UNEXPECTED |  | 
lm_head.bias              | UNEXPECTED |  | 
lm_head.layer_norm.weight | UNEXPECTED |  | 
lm_head.layer_norm.bias   | UNEXPECTED |  | 
lm_head.dense.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch,Training Loss,Validation Loss,Accuracy,Qwk,Mae,Rmse
1,0.383100,0.323477,0.370044,0.430394,0.911894,1.250551
2,0.288954,0.335634,0.409692,0.598298,0.775330,1.094641
3,0.276554,0.288058,0.475771,0.791060,0.634361,0.929213
4,0.238326,0.277655,0.440529,0.798954,0.651982,0.919682
5,0.221347,0.283523,0.431718,0.802807,0.660793,0.924460
6,0.211941,0.269816,0.466960,0.822454,0.612335,0.878023


Fold 5 Result: QWK=0.8225 | MAE=0.6123

FINAL CORN RESULTS (5-Fold CV)
ACCURACY: 0.4973 ± 0.0330
QWK: 0.8314 ± 0.0125
MAE: 0.5791 ± 0.0336
RMSE: 0.8624 ± 0.0237
